# Caderno 1. Gerar índice invertido

Gera dois índices invertidos. O primeiro considera todos os chunks. O segundo considera apenas os chunks relacionados à artigos ou à jurisprudência.

In [1]:
import json
import os
from tqdm import tqdm

In [2]:
REINDEXAR_INDICE_BM25_TODOS_CHUNKS = False
NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS = 'outputs/1 - indices_invertidos/indice_bm25_todos_chunks.pickle'

REINDEXAR_INDICE_BM25_APENAS_ART = False
NOME_ARQUIVO_INDICE_BM25_APENAS_ART = 'outputs/1 - indices_invertidos/indice_bm25_apenas_art.pickle'

# 1. Carregar as bases de dados de chunks e de questões

Carrega as bases de dados de questões (são as queries) e os chunks de pesquisa (são os documentos a serem pesquisados).

In [3]:
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

chunks_pesquisa = load_jsonl('inputs/chunks_pesquisa.jsonl')
questoes = load_jsonl('inputs/questoes.jsonl')

Filtra chunks_pesquisa para isolar apenas os chunks que são artigos completos (e os chunks de jurisprudência). Isso apenas será utilizado na criação do índice, em outra seção.

In [4]:
chunks_pesquisa_apenas_art = [c for c in chunks_pesquisa if c['TIPO'] == 'ART' or c['TIPO'] == 'JUR']

# 2. Cria índices invertidos para o campo TEXTO

In [5]:
from bm25 import IndiceInvertido, BM25, tokenizador_pt_remove_html

# Índice para todos os chunks
iidx_todos_chunks = IndiceInvertido(tokenizador_pt_remove_html)
if REINDEXAR_INDICE_BM25_TODOS_CHUNKS or not os.path.exists(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS):
    # Se for indexar a primeira vez:
    # Demora cerca de 35 minutos para indexar
    iidx_todos_chunks.adiciona_objetos(chunks_pesquisa, lambda obj: (obj['URN'], obj['TEXTO']))
    iidx_todos_chunks.to_pickle(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS)
else:
    # Se quiser recuperar de um arquivo:
    iidx_todos_chunks.from_pickle(NOME_ARQUIVO_INDICE_BM25_TODOS_CHUNKS)

# Índice para os chunks apenas de artigos
iidx_apenas_art = IndiceInvertido(tokenizador_pt_remove_html)
if REINDEXAR_INDICE_BM25_APENAS_ART or not os.path.exists(NOME_ARQUIVO_INDICE_BM25_APENAS_ART):
    # Se for indexar a primeira vez:
    # Demora cerca de 35 minutos para indexar
    iidx_apenas_art.adiciona_objetos(chunks_pesquisa_apenas_art, lambda obj: (obj['URN'], obj['TEXTO']))
    iidx_apenas_art.to_pickle(NOME_ARQUIVO_INDICE_BM25_APENAS_ART)
else:
    # Se quiser recuperar de um arquivo:
    iidx_apenas_art.from_pickle(NOME_ARQUIVO_INDICE_BM25_APENAS_ART)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\P_8454\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
100%|██████████████████████████████████████████████████████████████████████████████| 1078/1078 [00:39<00:00, 27.08it/s]
